In [2]:
from google.colab import files
uploaded = files.upload()

Saving data_set2.pkl to data_set2.pkl


Dataset importing - getDataSet.py generates the pickle file imported here



In [0]:
import _pickle as cPickle
f = open('data_set2.pkl','rb')
train_set, test_set, dicts = cPickle.load(f,encoding='utf-8')

In [0]:
train_lex, train_y, train_z = train_set
test_lex,  test_y, test_z  = test_set

In [0]:
MAX_LENGTH = len(max(train_lex, key=len))

Padding for variable sentence lengths

In [6]:
from keras.preprocessing.sequence import pad_sequences
train_sentences_X = pad_sequences(train_lex,maxlen=MAX_LENGTH,padding='post')
train_tags_y = pad_sequences(train_y,maxlen=MAX_LENGTH,padding='post')

Using TensorFlow backend.


In [0]:
test_sentences_X = pad_sequences(test_lex,maxlen=MAX_LENGTH,padding='post')
test_tags_y = pad_sequences(test_y,maxlen=MAX_LENGTH,padding='post')

In [0]:
train_tags_z = pad_sequences(train_z,maxlen=MAX_LENGTH,padding='post')
test_tags_z = pad_sequences(test_z,maxlen=MAX_LENGTH,padding='post')

Function for input to softmax layer

In [0]:
import numpy as np
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation, Dropout, Input
from keras.optimizers import Adam

Ignoring the accuracies obtained from padding - since including that doesn't give true accuracy

In [0]:
from keras import backend as K
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'float32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'float32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy

Softmax layer with 3 nodes (should be 2 for keyword and not keyword) but we need to consider the padded output too

In [0]:
from keras.models import Model
inputLayer = Input(shape=(MAX_LENGTH,),dtype='float32')
embedding = Embedding(len(dicts['words2idx'].keys())+1,128)(inputLayer)
hiddenLayer1 = LSTM(128,return_sequences=True)(embedding)
keywordDenseLayer = TimeDistributed(Dense(3,activation='softmax'),name='keyword')(hiddenLayer1)
model1 = Model(inputLayer, keywordDenseLayer)

In [0]:
model1.compile(loss = 'categorical_crossentropy',
             optimizer = Adam(0.1),
             metrics = ['accuracy',ignore_class_accuracy(0)])

In [45]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 226)               0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 226, 128)          30727552  
_________________________________________________________________
lstm_13 (LSTM)               (None, 226, 128)          131584    
_________________________________________________________________
keyword (TimeDistributed)    (None, 226, 3)            387       
Total params: 30,859,523
Trainable params: 30,859,523
Non-trainable params: 0
_________________________________________________________________


In [46]:
f = model1.fit(train_sentences_X,to_categorical(train_tags_y,3),batch_size=128,epochs=2,validation_split=0.2,shuffle=True)

Train on 62652 samples, validate on 15664 samples
Epoch 1/2
62652/62652 [==============================] - 713s 11ms/step - loss: 0.0948 - acc: 0.9804 - ignore_accuracy: 0.8310 - val_loss: 0.0273 - val_acc: 0.9905 - val_ignore_accuracy: 0.8577
Epoch 2/2
62652/62652 [==============================] - 706s 11ms/step - loss: 0.0248 - acc: 0.9912 - ignore_accuracy: 0.8660 - val_loss: 0.0240 - val_acc: 0.9917 - val_ignore_accuracy: 0.8721


In [48]:
scores = model1.evaluate(test_sentences_X, to_categorical(test_tags_y, 3))

33541/33541 [==============================] - 110s 3ms/step


In [49]:
print(scores)

[0.024201613425922388, 0.9916376266516882, 0.8707720033509236]


87% accuracy